### Filtering manually and statistically (Step 1)
manual deletions in meshlab might still be better

In [33]:
import open3d as o3d
import numpy as np

In [34]:
# Step 1: Load the PLY file
point_cloud = o3d.io.read_point_cloud("original_maze.ply") #week 1 maze
print('succeeded')

succeeded


In [40]:
"""visualise"""
# Add axis
axpoints = o3d.utility.Vector3dVector([[0, 0, 0], [1, 0, 0], [0, 1, 0]])
axis = o3d.geometry.TriangleMesh.create_coordinate_frame(
    size=10, origin=[0, 0, 0]
)

# Maze coord system: X-red, Y-green, Z-blue
# Global coord system: X-blue, Y-red, Z-green
#Show
o3d.visualization.draw_geometries([point_cloud,axis]) 

In [39]:
"""Transform to np and inspect"""
point_cloud_np = np.asarray(point_cloud.points) 
print('Shape: ', point_cloud_np.shape)
print('First point: ', point_cloud_np[0, :])


Shape:  (117207, 3)
First point:  [-11.54759979 -14.52369976 -14.12819958]


Analysis

In [37]:
print("X")
x=point_cloud_np[:, 2]
print("mean",np.mean(x))
print("std", np.std(x))
print("max",np.max(x))
print("min",np.min(x))
print("")

print("Y")
y=point_cloud_np[:, 0]
print("mean",np.mean(y))
print("std", np.std(y))
print("max",np.max(y))
print("min",np.min(y))
print("")


print("Z")
z=point_cloud_np[:, 1]
print("mean",np.mean(z))
print("std", np.std(z))
print("max",np.max(z))
print("min",np.min(z))


X
mean -2.8372596370975196
std 4.397610378525984
max 20.4419002532959
min -20.409099578857422

Y
mean -3.494563178628163
std 6.153619353848993
max 18.94409942626953
min -31.097999572753906

Z
mean -2.612891753537297
std 3.9294963400404943
max 0.3499389886856079
min -25.049999237060547


In [ ]:
"""1. Filter manually"""
# Filter z (dimension 1)
z_upper = 0
z_lower = -2.0 #removes ceiling points
filtered_points = point_cloud_np[(point_cloud_np[:, 1] < z_upper) & (point_cloud_np[:, 1] > z_lower)]

# # Filter y (dimension 0)
# y_upper = 4.8
# y_lower = -13
# filtered_points = filtered_points[(filtered_points[:, 0] < y_upper) & (filtered_points[:, 0] > y_lower)]

# # Filter x (dimension 2)
# x_upper = 3
# x_lower = -11.6
# filtered_points = filtered_points[(filtered_points[:, 2] < x_upper) & (filtered_points[:, 2] > x_lower)]

In [38]:
"""Visualise"""
# transform to open3d pc again
filtered_pcd = o3d.geometry.PointCloud()
filtered_pcd.points = o3d.utility.Vector3dVector(filtered_points)

o3d.visualization.draw_geometries([filtered_pcd]) 

In [42]:
"""2. Filter out isolated points / outliers with o3d function"""
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(filtered_points)

### statistical outlier filtering
pcd_clean, ind = pcd.remove_statistical_outlier(nb_neighbors=10, std_ratio=2.0)
# nb_neighbors: Number of neighbors considered 
# Points further away than mean distance + std_ratio*std_dev are considered outliers



### radius filtering << didnt use
#pcd_clean, ind = pcd.remove_radius_outlier(nb_points=5, radius=0.1)
# radius: radius to consider for the neighbors
# min_points: minimum number of neighbors required within that radius

o3d.visualization.draw_geometries([pcd_clean])

In [43]:
"""Save filtered pointcloud"""
o3d.io.write_point_cloud("filtered_pointcloud_week1.ply", pcd_clean)

True